In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def plot_var(xinf, taux, ax, x_range=[-150,101], col='k', label=''):
    n = 1013
    if x_range[0] < 0 or np.diff(np.log10(x_range)) < 3:
        x = np.linspace(x_range[0],x_range[1],n)
        var_name = 'Voltage (mV)'
        mode = 'lin'
    else:
        x = np.logspace(np.log10(x_range[0]),np.log10(x_range[1]),n)
        var_name = 'Calcium (mM)'
        mode = 'log'
    if mode == 'lin':
        ax[0].plot(x,xinf(x),color=col,lw=1)
    else:
        ax[0].semilogx(x,xinf(x),color=col,lw=1)
    ax[0].set_ylabel(r'$x_\infty$')
    if mode == 'lin':
        ax[1].plot(x,taux(x),color=col,lw=1,label=label)
    else:
        ax[1].semilogx(x,taux(x),color=col,lw=1,label=label)
    if label != '':
        ax[1].legend(loc='best')
    ax[1].set_xlabel(var_name)
    ax[1].set_ylabel(r'$\tau$ (ms)')

In [ ]:
def trap0(v,th,a,q):
    y = a*q*np.ones(v.shape)
    idx, = np.where(np.abs(v-th)>1e-6)
    y[idx] = a * (v[idx]-th) / (1-np.exp(-(v[idx]-th)/q))
    return y

In [ ]:
TEMPERATURE = 34
R = 8.313424
FARADAY = 96.48533212

In [ ]:
# nax: transient sodium current, used in all compartments
# I = gbar * m^3 * h * (V-ENa)
tha = -30
thinf = -50
sh = 0
Ra = 0.4
Rb = 0.14
Rd = 0.03
Rg = 0.01
qa = 7.2
qd = 1.5
qg = 1.5
q10 = 2
qinf = 4
thi1 = -45
thi2 = -45
qt_nax = q10 ** ((TEMPERATURE - 24.)/10.)
ma_nax = lambda v: trap0(v, tha+sh, Ra, qa)
mb_nax = lambda v: trap0(-v, -tha-sh, Rb, qa)
minf_nax = lambda v: ma_nax(v)/(ma_nax(v) + mb_nax(v))
taum_nax = lambda v: (1/(ma_nax(v) + mb_nax(v))) / qt_nax
ha_nax = lambda v: trap0(v, thi1+sh, Rd, qd)
hb_nax = lambda v: trap0(-v, -thi2-sh, Rg, qg)
hinf_nax = lambda v: 1 / (1+np.exp((v-thinf-sh)/qinf))
tauh_nax = lambda v: (1/(ha_nax(v) + hb_nax(v))) / qt_nax

In [ ]:
# nap: persistent sodium current, used in somatic and apical compartments
# I = gbar * n^3 * (V-ENa)
vhalf = -60.4
K = 2
ninf_nap = lambda v: 1 / (1 + (np.exp(vhalf - v)/K))
taun_nap = lambda v: 10. + np.zeros(v.shape)

In [ ]:
# kdr: delayed rectifier potassium current, used in all compartments
# I = gbar * n * (V-EK)
zetan_kdr = -3
vhalfn_kdr = 13
gmn_kdr = 0.7
a0n_kdr = 0.02
q10 = 1
qt_kdr = q10 ** ((TEMPERATURE-24)/10)
na_kdr = lambda v: np.exp(1.e-3*zetan_kdr*(v - vhalfn_kdr)*9.648e4/(8.315*(273.16 + TEMPERATURE)))
nb_kdr = lambda v: np.exp(1.e-3*zetan_kdr*gmn_kdr*(v - vhalfn_kdr)*9.648e4/(8.315*(273.16 + TEMPERATURE))) 
ninf_kdr = lambda v: 1 / (1 + na_kdr(v))
taun_kdr = lambda v: nb_kdr(v) / (qt_kdr * a0n_kdr * (1+na_kdr(v)))

In [ ]:
# km: muscarinic potassium current, used in somatic and axonal compartments
# I = gbar * m * (V-EK)
b0 = 60
a0t = 0.003
vhalfl = -40
vhalft = -42
sh_km = 0
kl = -10
zetat = 7
gmt = 0.4
q10 = 5
qt_km = q10 ** ((TEMPERATURE-35)/10)
ma_km = lambda v: np.exp(0.0378 * zetat * (v - vhalft - sh_km)) 
mb_km = lambda v: np.exp(0.0378 * zetat * gmt * (v - vhalft - sh_km)) 
minf_km = lambda v: (1. / (1 + np.exp((v - vhalfl - sh)/kl)))
taum_km = lambda v: b0 + mb_km(v) / (a0t * (1 + ma_km(v)))

In [ ]:
# kap: A-type potassium current, used in proximal dendritic compartments
# I = gbar * n * l * (V-EK)
pw = -1
tq = -40
qq = 5
gmn_kap = 0.55
a0n_kap = 0.05
zetan_kap = -1.5
zetal_kap = 3
vhalfn_kap = 11
vhalfl_kap = -56
lmin = 2
qtl = 1
q10 = 5
qt_kap = q10 ** ((TEMPERATURE - 24.)/10.)
zeta_kap = lambda v: zetan_kap + pw / (1+np.exp((v-tq)/qq))
na_kap = lambda v: np.exp(1e-3 * zeta_kap(v) * (v-vhalfn_kap) * 9.648e4 / (8.315*(273.16+TEMPERATURE)))
nb_kap = lambda v: np.exp(1e-3 * zeta_kap(v) * gmn_kap * (v-vhalfn_kap) * 9.648e4 / (8.315*(273.16+TEMPERATURE)))
ninf_kap = lambda v: 1 / (1 + na_kap(v))
taun_kap = lambda v: nb_kap(v) / (qt_kap * a0n_kap * (1+na_kap(v)))
la_kap = lambda v: np.exp(1e-3 * zetal_kap * (v-vhalfl_kap) * 9.648e4 / (8.315*(273.16+TEMPERATURE)))
linf_kap = lambda v: 1 / (1 + la_kap(v))
def taul_kap(v):
    tau = 0.26 * (v+50) / qtl
    tau[tau < lmin/qtl] = lmin / qtl
    return tau

In [ ]:
# kad: A-type potassium current, used in distal dendritic compartments
# I = gbar * n * l * (V-EK)
pw = -1
tq = -40
qq = 5
gmn_kad = 0.39
a0n_kad = 0.1
zetan_kad = -1.8
zetal_kad = 3
vhalfn_kad = -1
vhalfl_kad = -56
lmin = 2
qtl = 1
q10 = 5
qt_kad = q10 ** ((TEMPERATURE - 24.)/10.)
zeta_kad = lambda v: zetan_kad + pw / (1+np.exp((v-tq)/qq))
na_kad = lambda v: np.exp(1e-3 * zeta_kad(v) * (v-vhalfn_kad) * 9.648e4 / (8.315*(273.16+TEMPERATURE)))
nb_kad = lambda v: np.exp(1e-3 * zeta_kad(v) * gmn_kad * (v-vhalfn_kad) * 9.648e4 / (8.315*(273.16+TEMPERATURE)))
ninf_kad = lambda v: 1 / (1 + na_kad(v))
taun_kad = lambda v: nb_kad(v) / (qt_kad * a0n_kad * (1+na_kad(v)))
la_kad = lambda v: np.exp(1e-3 * zetal_kad * (v-vhalfl_kad) * 9.648e4 / (8.315*(273.16+TEMPERATURE)))
linf_kad = lambda v: 1 / (1 + la_kad(v))
def taul_kad(v):
    tau = 0.26 * (v+50) / qtl
    tau[tau < lmin/qtl] = lmin / qtl
    return tau

In [ ]:
# cagk: calcium-activated potassium current, used in somatic and dendritic compartments
# I = gbar * o * (V-EK)
k1 = 0.48e-3
k2 = 0.13e-6
d1 = 0.84
d2 = 1.0
abar = 0.28
bbar = 0.48
exp1 = lambda k,d,v: k * np.exp(-2 * d * FARADAY * v / R / (273.15 + TEMPERATURE))
oa_cagk = lambda v,cai: cai * abar / (cai + exp1(k1,d1,v))
ob_cagk = lambda v,cai: bbar / (1 + cai / exp1(k2,d2,v))
oinf_cagk = lambda v,cai: oa_cagk(v,cai) / (oa_cagk(v,cai) + ob_cagk(v,cai))
tauo_cagk = lambda v,cai: 1 / (oa_cagk(v,cai) + ob_cagk(v,cai))

In [ ]:
# kca: slow calcium-activated potassium current, used in somatic and dendritic compartments
# I = gbar * m^3 * (V-EK)
cac = 0.00035
beta = 0.03
taumin = 0.5
tadj = 3 ** ((TEMPERATURE-22.0)/10)
car = lambda cai,n=1: (cai/cac)**4 + np.zeros(n)
minf_kca = lambda v,cai: car(cai,v.shape[0]) / (1 + car(cai,v.shape[0]))
def taum_kca(v, cai):
    tau =  1 / beta / (1 + car(cai,v.shape[0])) / tadj
    tau[tau < taumin] = taumin
    return tau

In [ ]:
# cal: L-type calcium channel, used in somatic and dendritic compartments
q10 = 5
qt_cal = q10 ** ((TEMPERATURE - 25)/10)
ma_cal = lambda v: 15.69*(-1.0*v+81.5)/(np.exp((-1.0*v+81.5)/10.0)-1.0)
mb_cal = lambda v: 1 / ((ma_cal(v) + 0.29*np.exp(-v/10.86)))
minf_cal = lambda v: ma_cal(v) * mb_cal(v)
def taum_cal(v):
    a0m = 0.1
    vhalfm = 4
    zetam = 2
    gmm = 0.1
    mmin = 0.2
    betmt = np.exp(0.0378 * zetam * gmm * (v-vhalfm))
    alpmt = np.exp(0.0378 * zetam * (v-vhalfm))
    tau = betmt / (qt_cal * a0m * (1+alpmt))
    tau[tau<mmin/qt_cal] = mmin / qt_cal
    return tau

In [ ]:
# cat: T-type calcium channel, used in somatic and dendritic compartments
q10 = 5
qt_cat = q10 ** ((TEMPERATURE-25)/10)
ma_cat = lambda v: 0.2*(-1.0*v+19.26)/(np.exp((-1.0*v+19.26)/10.0)-1.0)
mb_cat = lambda v: 0.009*np.exp(-v/22.03)
minf_cat = lambda v: ma_cat(v) / (ma_cat(v) + mb_cat(v))
def taum_cat(v):
    a0m = 0.04
    mmin = 0.2
    zetam = 2
    gmm = 0.1
    vhalfm = -28
    alpmt = lambda v: np.exp(0.0378*zetam*(v-vhalfm))
    betmt = lambda v: np.exp(0.0378*zetam*gmm*(v-vhalfm))
    tau = betmt(v) / (qt_cat * a0m * (1+alpmt(v)))
    tau[tau < mmin] = mmin
    return tau

ha_cat = lambda v: 1e-6*np.exp(-v/16.26)
hb_cat = lambda v: 1/(np.exp((-v+29.79)/10.)+1.)
hinf_cat = lambda v: ha_cat(v) / (ha_cat(v) + hb_cat(v))
def tauh_cat(v):
    a0h = 0.015
    hmin = 10
    zetah = 3.5
    gmh = 0.6
    vhalfh = -75
    alph = lambda v: np.exp(0.0378*zetah*(v-vhalfh))
    beth = lambda v: np.exp(0.0378*zetah*gmh*(v-vhalfh))
    tau = beth(v) / (a0h*(1+alph(v)))
    tau[tau < hmin] = hmin
    return tau

In [ ]:
# can: N-type calcium channel, used in somatic and dendritic compartments
q10 = 5
qt_can = q10 ** ((TEMPERATURE-25)/10)
ma_can = lambda v: 0.1967*(-1.0*v+19.88)/(np.exp((-1.0*v+19.88)/10.0)-1.0)
mb_can = lambda v: 1 / (ma_cat(v) + 0.046*np.exp(-v/20.73))
minf_can = lambda v: ma_can(v) * mb_can(v)
def taum_can(v):
    zetam = 2
    vhalfm = -14
    gmm = 0.1
    mmin = 0.2
    a0m = 0.03
    alpmt = lambda v: np.exp(0.0378*zetam*(v-vhalfm))
    betmt = lambda v: np.exp(0.0378*zetam*gmm*(v-vhalfm))
    tau = betmt(v) / (qt_can*a0m*(1 + alpmt(v)))
    tau[tau < mmin/qt_can] = mmin/qt_can
    return tau

ha_can = lambda v: 1.6e-4*np.exp(-v/48.4)
hb_can = lambda v: 1/(ha_can(v) + 1/(np.exp((-v+39.0)/10.)+1.))
hinf_can = lambda v: ha_can(v) * hb_can(v)
tauh_can = lambda v: 80 + np.zeros(v.shape)

### Sodium currents

In [ ]:
fig,ax = plt.subplots(2, 1, sharex=True, figsize=(8,7))
plot_var(minf_nax, taum_nax, ax, col=[0,0,0], label='nax m')
plot_var(hinf_nax, tauh_nax, ax, col=[.7,0,0], label='nax h')
plot_var(ninf_nap, taun_nap, ax, col=[0,.7,0], label='nap n')

### Potassium currents

In [ ]:
fig,ax = plt.subplots(2, 1, sharex=True, figsize=(8,7))
plot_var(ninf_kdr, taun_kdr, ax, col=[0,0,0], label='kdr n')
plot_var(minf_km, taum_km, ax, col=[1,0,.7], label='km m')
plot_var(ninf_kap, taun_kap, ax, col=[1,0,0], label='kap n')
plot_var(linf_kap, taul_kap, ax, col=[1,.7,0], label='kap l')
plot_var(ninf_kad, taun_kad, ax, col=[0,0,1], label='kad n')
plot_var(linf_kad, taul_kad, ax, col=[0,.7,1], label='kad l')

### Calcium-dependent potassium currents

In [ ]:
fig,ax = plt.subplots(2, 1, sharex=True, figsize=(8,7))
cmap = [[0,0,0], [.4,.4,.4], [.7,.7,.7]]
for i,cai in enumerate((50, 500, 5000)):
    plot_var(lambda v: oinf_cagk(v,cai*1e-6), lambda v: tauo_cagk(v,cai*1e-6), ax, \
             col=cmap[i], label='cagk o {}e-6'.format(cai))
cmap = [[1,0,0], [1,.4,.4], [1,.7,.7]]
for i,cai in enumerate((50, 500, 5000)):
    plot_var(lambda v: minf_kca(v,cai*1e-6), lambda v: taum_kca(v,cai*1e-6), ax, \
             col=cmap[i], label='kca m {}e-6'.format(cai))

### Calcium currents

In [ ]:
fig,ax = plt.subplots(2, 1, sharex=True, figsize=(8,7))
plot_var(minf_cal, taum_cal, ax, col=[0,0,0], label='cal m')
plot_var(minf_cat, taum_cat, ax, col=[1,0,0], label='cat m')
plot_var(hinf_cat, tauh_cat, ax, col=[1,.5,0], label='cat h')
plot_var(minf_can, taum_can, ax, col=[0,0,1], label='can m')
plot_var(hinf_can, tauh_can, ax, col=[0,.5,1], label='can h')